In [ ]:
### Using DOPA effect as BL since this appears to be a normal healthy DA response SMS 15FEB2024
import os
from netpyne import specs, sim


# Create a folder to save the results if it doesn't exist
results_dir = 'results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

### Set variables for Individul Patient Loop

## equations designed as % changes from baseline = 1

## Variables below should increase at similar rates as a fnx of amph. mediated dopa release
Nap_pyr_soma=[1, 1.05, 1.1, 1.1, 1.1, 1.1];
Hva_pyr_soma=[1, 1.05, 1.1, 1.1, 1.1, 1.1];
Iks_pyr_soma=[1, 1.05, 1.1, 1.1, 1.1, 1.1];
Nap_pyr_bas=[1, 1.05, 1.1, 1.1, 1.1, 1.1];
Hva_pyr_bas=[1, 1.05, 1.1, 1.1, 1.1, 1.1];
Iks_pyr_bas=[1, 1.05, 1.1, 1.1, 1.1, 1.1];
Nap_pyr_trunk=[1, 1.05, 1.1, 1.1, 1.1, 1.1];
Hva_pyr_trunk=[1, 1.05, 1.1, 1.1, 1.1, 1.1];
Iks_pyr_trunk=[1, 1.05, 1.1, 1.1, 1.1, 1.1];
Hva_pyr_apical=[1, 1.05, 1.1, 1.1, 1.1, 1.1];
Iks_pyr_apical=[1, 1.05, 1.1, 1.1, 1.1, 1.1];

#Connecticity of the circuit should change as a fnx of different patient (ie. schizo lower synatpic connectivity)
exex=[1, 1, 1, 1, 1, 1];
exin=[1, 1, 1, 1, 1, 1];
inex=[1, 1, 1, 1, 1, 1];
exexInterAssem=[1, 1, 1, 1, 1, 1];
inin=[1, 1, 1, 1, 1, 1];

#Strength of AMPA and NMDA conductions will be altered according to Goldman-Rakic 1998 publication
AMPAgEE=[.75, .75, .85, .85, .85, .85];
NMDAgEE=[.75, .75, .85, .85, .85, .85];
AMPAgEE_assem=[.75, .75, .85, .85, .85, .85];
NMDAgEE_assem=[.75, .75, .85, .85, .85, .85];
AMPAgEI=[1, 1, 1.05, 1.075, 1.1, 1.2];
NMDAgEI=[1, 1, 1.05, 1.075, 1.1, 1.2];
GABAgIE=[1, 1, 1, 1, 1, 1];
GABAgII=[1, 1, 1, 1, 1, 1];


# Define the analyzeRaster function outside the loop
def analyzeRaster(simData, patient):
    spike_times = simData['spkt']
    spike_cells = simData['spkid']
    duration = 0
    burst_count = 0
    in_burst = False

    for i in range(len(spike_times)):
        if spike_times[i] >= 5000:  # Start analysis at 5 seconds
            if not in_burst:
                in_burst = True
                burst_count += 1
            duration += 1
        else:
            if in_burst:
                in_burst = False

    # Calculate burst frequency
    burst_frequency = burst_count / (simConfig.duration - 5000)  # Duration in ms

    # Save the data for each patient
    output_dir = 'burstdata'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    with open(os.path.join(output_dir, f'burstdata_patient_{patient}.txt'), 'w') as file:
        file.write(f'Patient {patient}:\n')
        file.write(f'Burst Duration starting at 5 seconds: {duration} ms\n')
        file.write(f'Burst Frequency: {burst_frequency} Hz\n')
    
    # Print the burst duration and frequency
    print(f'Patient {patient}:')
    print(f'Burst Duration starting at 5 seconds: {duration} ms')
    print(f'Burst Frequency: {burst_frequency} Hz')
    
    # Return the burst duration and frequency
    return duration, burst_frequency


for patient in (0,1,2,3,4,5):

        Dt = 0.05
        Tfinal = 30000

        # Network parameters
        netParams = specs.NetParams()  # object of class NetParams to store the network parameters
        netParams.defaultThreshold = 0.0

        ###############################################################################
        ## Cell types
        ###############################################################################
        # Pyramidal cell
        PyrCell = {'secs':{}, 'globals': {'ko0_k_ion': 3.82, 'ki0_k_ion': 160, 'cao0_ca_ion': 2, 'cai0_ca_ion': 50e-6}}
        # soma
        PyrCell['secs']['soma'] = {'geom': {}, 'mechs': {}}
        PyrCell['secs']['soma']['geom'] = {'diam': 23, 'L': 23, 'cm': 1.2, 'Ra': 150, 'nseg': 1}
        PyrCell['secs']['soma']['mechs'] = {
            'pas': {'g': 3.33e-5, 'e': -70},
            'kdyn': {},
            'Naf': {'gnafbar': 0.086},#same
            'Nap': {'gnapbar': [0.00154*Nap_pyr_soma[patient]]},#reduced to 75% of baseline #DopaEffect
            'Hva': {'ghvabar': [0.000272*Hva_pyr_soma[patient]]},#reduced to 80% of baseline (as in original model) #DopaEffect
            'kdr': {'gkdrbar': 0.0338},#same
            'IKs': {'gKsbar': [0.000105*Iks_pyr_soma[patient]]},# reducted by 75% of baseline (50% of baseline used in original mode) #DopaEffect
            'iC': {'gkcbar': 0.0022},#same
            'cadyn': {'CAF': 385.948e-9, 'tca': 250}}#same

        # basal dendrite
        PyrCell['secs']['Bdend'] = {'geom': {}, 'mechs': {}}
        PyrCell['secs']['Bdend']['geom'] = {'diam': 16.0, 'L': 150.0, 'cm': 2.3, 'Ra': 150, 'nseg': 1}
        PyrCell['secs']['Bdend']['topol'] = {'parentSec': 'soma', 'parentX': 0, 'childX': 0}
        PyrCell['secs']['Bdend']['mechs'] = {
            'pas': {'g': 6.39e-5, 'e': -70},
            'kdyn': {},
            'Naf': {'gnafbar': 0.028},
            'Nap': {'gnapbar': [0.0007*Nap_pyr_bas[patient]]},
            'Hva': {'ghvabar': [0.00056*Hva_pyr_bas[patient]]},
            'kdr': {'gkdrbar': 0.0092},
            'IKs': {'gKsbar': [0.00018*Iks_pyr_bas[patient]]},
            'iC': {'gkcbar': 0.0038},
            'cadyn': {'CAF': 964.87e-9, 'tca': 120}}

        # apical proximal dendrite
        PyrCell['secs']['Adend_proximal'] = {'geom': {}, 'mechs': {}}
        PyrCell['secs']['Adend_proximal']['geom'] = {'diam': 2.6, 'L': 400.0, 'cm': 2.3, 'Ra': 150, 'nseg': 1}
        PyrCell['secs']['Adend_proximal']['topol'] = {'parentSec': 'soma', 'parentX': 1, 'childX': 0}
        PyrCell['secs']['Adend_proximal']['mechs'] = {
            'pas': {'g': 6.39e-5, 'e': -70},
            'kdyn': {},
            'Naf': {'gnafbar': 0.028},
            'Nap': {'gnapbar': [0.0007*Nap_pyr_trunk[patient]]},
            'Hva': {'ghvabar': [0.00056*Hva_pyr_trunk[patient]]},
            'kdr': {'gkdrbar': 0.0092},
            'IKs': {'gKsbar': [0.00018*Iks_pyr_trunk[patient]]},
            'iC': {'gkcbar': 0.0038},
            'cadyn': {'CAF': 964.87e-9, 'tca': 120}}

        # apical distal dendrite
        PyrCell['secs']['Adend_distal'] = {'geom': {}, 'mechs': {}}
        PyrCell['secs']['Adend_distal']['geom'] = {'diam': 2.6, 'L': 400.0, 'cm': 2.3, 'Ra': 150, 'nseg': 1}
        PyrCell['secs']['Adend_distal']['topol'] = {'parentSec': 'Adend_proximal', 'parentX': 1, 'childX': 0}
        PyrCell['secs']['Adend_distal']['mechs'] = {
            'pas': {'g': 6.39e-5, 'e': -70},
            'kdyn': {},
            'Naf': {'gnafbar': 0.028},
            'Hva': {'ghvabar': [0.00017*Hva_pyr_apical[patient]]},
            'kdr': {'gkdrbar': 0.0092},
            'IKs': {'gKsbar': [0.00018*Iks_pyr_apical[patient]]},
            'iC': {'gkcbar': 0.0022},
            'cadyn': {'CAF': 964.87e-9, 'tca': 80}}

        netParams.cellParams['PyrCell'] = PyrCell

        ###############################################################################
        # Inhibitory cell
        InhCell = {'secs':{}}
        # soma
        InhCell['secs']['soma'] = {'geom': {}, 'mechs': {}}
        InhCell['secs']['soma']['geom'] = {'diam': 15, 'L': 15, 'cm': 1.0, 'Ra': 150, 'nseg': 1}
        InhCell['secs']['soma']['mechs'] = {
            'pas': {'g': 1.0e-5, 'e': -68},
            'kdyn': {},
            'Naf': {'gnafbar': 0.100},
            'kdr': {'gkdrbar': 0.040}}

        # dendrite
        InhCell['secs']['dend'] = {'geom': {}, 'mechs': {}}
        InhCell['secs']['dend']['geom'] = {'diam': 10, 'L': 150, 'cm': 1.92, 'Ra': 150, 'nseg': 1}
        InhCell['secs']['dend']['topol'] = {'parentSec': 'soma', 'parentX': 1, 'childX': 0}
        InhCell['secs']['dend']['mechs'] = {
            'pas': {'g': 1.92e-5, 'e': -68},
            'kdyn': {},
            'Naf': {'gnafbar': 0.020},
            'kdr': {'gkdrbar': 0.008}}

        netParams.cellParams['InhCell'] = InhCell


        ###############################################################################
        ## Synaptic mechs
        ###############################################################################

        netParams.synMechParams['AMPA'] = {'mod': 'Exp2Syn', 'tau1': 0.5, 'tau2': 2.5, 'e': 0}
        netParams.synMechParams['NMDA'] = {'mod': 'Exp2SynNMDA', 'tau1': 10, 'tau2': 250.0, 'e': 0}
        netParams.synMechParams['GABA'] = {'mod': 'ExpSyn', 'tau': 1.0, 'e': -75}


        ###############################################################################
        ## Populations
        ###############################################################################
        # Structuring excitatory assemblies
        Nassemblies = 2
        Nexc = 10
        Nexc_total = Nassemblies * Nexc       # without overlap between assemblies
        for na in range(Nassemblies):
            ### Populations
            netParams.popParams['Pyr_Assembly'+str(na+1)] = {'numCells': Nexc, 'cellType': 'PyrCell'}

        # Inhibitory
        Ninh = 10                             # global inhibitory neurons 
        netParams.popParams['InhPop'] = {'numCells': Ninh, 'cellType': 'InhCell'}


        ###############################################################################
        ## Connections
        ###############################################################################
        for na in range(Nassemblies):
            ## Intra-assembly
            # pyr-to-pyr AMPA/NMDA, to basal dendrite
            netParams.connParams['PYR->PYR(intra)_AMPA/NMDA_Assembly'+str(na+1)] = {
                'preConds': {'pop': 'Pyr_Assembly'+str(na+1)}, 
                'postConds': {'pop': 'Pyr_Assembly'+str(na+1)},
                'connFunc': 'fullConn',
                'weight': [2.0e-3*AMPAgEE[patient], 0.56e-3*NMDAgEE[patient]],
                'delay': 'uniform(2,4)',
                'sec': ['Bdend','Adend_proximal'],   # synapses sampled at these regions
                'loc': 0.5,
                'synMech': ['AMPA','NMDA']}

            # pyr-to-inh AMPA/NMDA, to dendrite
            netParams.connParams['PYR->INH_AMPA/NMDA_Assembly'+str(na+1)] = {
                'preConds': {'pop': 'Pyr_Assembly'+str(na+1)}, 
                'postConds': {'pop': 'InhPop'},
                'connFunc': 'fullConn',
                'weight': [0.6*value for value in [2.0e-3*AMPAgEI[patient], 0.56e-3*NMDAgEI[patient]]],
                'delay': 'uniform(2,4)',
                'sec': 'dend',
                'loc': 0.5,
                'synMech': ['AMPA','NMDA']}
            
            # inh-to-pyr GABA, to soma
            netParams.connParams['INH->PYR_GABA_Assembly'+str(na+1)] = {
                'preConds': {'pop': 'InhPop'},
                'postConds': {'pop': 'Pyr_Assembly'+str(na+1)}, 
                'connFunc': 'fullConn',
                'weight': 7.8e-3*GABAgIE[patient],
                'delay': 'uniform(2,4)',
                'sec': 'soma',
                'loc': 0.5,
                'synMech': 'GABA'}
            
            ## Inter-assembly
            for nb in range(Nassemblies):
                if nb != na:
                    # pyr-to-pyr AMPA/NMDA, to basal dendrite
                    netParams.connParams['PYR->PYR(inter)_AMPA/NMDA_Assembly'+str(na+1)+'_to_Assembly'+str(nb+1)] = {
                        'preConds': {'pop': 'Pyr_Assembly'+str(na+1)}, 
                        'postConds': {'pop': 'Pyr_Assembly'+str(nb+1)},
                        'connFunc': 'fullConn',
                        'weight': [0.1*value for value in [2.0e-3*AMPAgEE_assem[patient], 0.56e-3*NMDAgEE_assem[patient]]],
                        'delay': 'uniform(2,4)',
                        'sec': ['Bdend','Adend_proximal'],   # synapses sampled at these regions
                        'loc': 0.5,
                        'synMech': ['AMPA','NMDA']}
                
        # inh-to-inh GABA, to soma
        netParams.connParams['INH->INH_GABA'] = {
            'preConds': {'pop': 'InhPop'},
            'postConds': {'pop': 'InhPop'}, 
            'connFunc': 'fullConn',
            'weight': 0.65e-3*GABAgII[patient],
            'delay': 'uniform(2,4)',
            'sec': 'soma',
            'loc': 0.5,
            'synMech': 'GABA'}
                    

        ###############################################################################
        ## Background stimulation
        ###############################################################################
        ### Source Stimulation - Spontaneous firing
        netParams.stimSourceParams['NetStim_PYR_exc']  = {'type': 'NetStim', 'interval': 1.0, 'start': 0, 'noise': 1, 'seed': 1}
        netParams.stimSourceParams['NetStim_PYR_exc2'] = {'type': 'NetStim', 'interval': 5.0, 'start': 0, 'noise': 1, 'seed': 2}
        netParams.stimSourceParams['NetStim_PYR_inh']  = {'type': 'NetStim', 'interval': 1.0, 'start': 0, 'noise': 1, 'seed': 3}
        netParams.stimSourceParams['NetStim_INH_exc']  = {'type': 'NetStim', 'interval': 1.0, 'start': 0, 'noise': 1, 'seed': 4}
        netParams.stimSourceParams['NetStim_INH_exc2'] = {'type': 'NetStim', 'interval': 5.0, 'start': 0, 'noise': 1, 'seed': 5}
        netParams.stimSourceParams['NetStim_INH_inh']  = {'type': 'NetStim', 'interval': 1.0, 'start': 0, 'noise': 1, 'seed': 6}

        ### Target Stimulation - Spontaneous firing
        for na in range(Nassemblies):
            # exc to PYR
            netParams.stimTargetParams['NetStim_PYR_exc->PYR_Assembly'+str(na+1)] = {
                'source': 'NetStim_PYR_exc',
                'conds': {'pop': 'Pyr_Assembly'+str(na+1)},
                'sec': 'soma',
                'loc': 0.5,
                'weight': 1.0e-3,
                'delay': 2*Dt,
                'synMech': 'AMPA'}

            netParams.stimTargetParams['NetStim_PYR_exc->PYR_Assembly_NMDA'+str(na+1)] = {
                'source': 'NetStim_PYR_exc2',
                'conds': {'pop': 'Pyr_Assembly'+str(na+1)},
                'sec': 'soma',
                'loc': 0.5,
                'weight': 0.1e-3,
                'delay': 2*Dt,
                'synMech': 'NMDA'}

            # inh to PYR
            netParams.stimTargetParams['NetStim_PYR_inh->PYR_Assembly'+str(na+1)] = {
                'source': 'NetStim_PYR_inh',
                'conds': {'pop': 'Pyr_Assembly'+str(na+1)},
                'sec': 'soma',
                'loc': 0.5,
                'weight': 7.5e-3,
                'delay': 2*Dt,
                'synMech': 'GABA'}

        # exc to INH
        netParams.stimTargetParams['NetStim_INH_exc->INH'] = {
            'source': 'NetStim_INH_exc',
            'conds': {'pop': 'InhPop'},
            'sec': 'soma',
            'loc': 0.5,
            'weight': 0.55e-3,
            'delay': 2*Dt,
            'synMech': 'AMPA'}

        netParams.stimTargetParams['NetStim_INH_exc->INH_NMDA'] = {
            'source': 'NetStim_INH_exc2',
            'conds': {'pop': 'InhPop'},
            'sec': 'soma',
            'loc': 0.5,
            'weight': 0.055e-3,
            'delay': 2*Dt,
            'synMech': 'NMDA'}

        # inh to INH
        netParams.stimTargetParams['NetStim_INH_inh->INH'] = {
            'source': 'NetStim_INH_inh',
            'conds': {'pop': 'InhPop'},
            'sec': 'soma',
            'loc': 0.5,
            'weight': 1.25e-3,
            'delay': 2*Dt,
            'synMech': 'GABA'}


        ###############################################################################
        ## Assembly recall
        ###############################################################################
        netParams.stimSourceParams['IClamp'] =  {'type': 'IClamp', 'del': Tfinal/2, 'dur': 100, 'amp': 1.00}
        netParams.stimTargetParams['IClamp->Pyr_Assembly1'] = {
                'source': 'IClamp',
                'sec': 'soma',
                'loc': 0.5,
                'conds': {'pop':'Pyr_Assembly1'}}


    
        
        ###############################################################################
        # Simulation options
        ###############################################################################
        simConfig = specs.SimConfig()

        simConfig.duration = Tfinal
        simConfig.dt = Dt
        simConfig.hParams = {'v_init': -66,'celsius': 23}
        simConfig.verbose = False
        simConfig.recordTraces = {'V_soma':{'sec':'soma','loc':0.5,'var':'v'}}
        #                          'ko_soma': {'sec': 'soma', 'loc': 0.5, 'var': 'ko'},
        #                          'cai_basal':{'sec':'Bdend','loc':0.5,'var':'cai'},
        #                          'ko_apical': {'sec': 'Adend_distal', 'loc': 0.5, 'var': 'ko'},
        #                          'synapse_inmda':{'sec':'soma','loc':0.5,'synMech':'NMDA','var':'i'},}  # Dict with traces to record
        simConfig.recordStim = True
        simConfig.recordStep = 0.1        # Step size in ms to save data (eg. V traces, LFP, etc)
        simConfig.filename = '00'         # Set file output name
        #simConfig.saveJson = True

        #simConfig.analysis['plotTraces'] = {'include': ['all'], 'saveFig': True}
        #simConfig.analysis['plotRaster'] = {'saveFig': True}
        
        # Update simConfig to save figures with patient flags
        simConfig.analysis['plotTraces'] = {'include': ['all'], 'saveFig': 'traces_pat{0}.png'.format(patient)}
        simConfig.analysis['plotRaster'] = {'saveFig': 'raster_pat{0}.png'.format(patient)}
       


        # Add this block of code inside the loop
        simConfig.recordTraces = {'V_soma':{'sec':'soma','loc':0.5,'var':'v'}}
        simConfig.recordStim = True
        simConfig.recordStep = 0.1
        simConfig.recordSpikes = 'all'  # Record spike times for all cells

        def analyzeRaster(simData, patient):
        # Your custom analysis function for calculating burst duration and frequency
            
            
            spike_times = simData['spkt']
            spike_cells = simData['spkid']
            duration = 0
            burst_count = 0
            in_burst = False

            for i in range(len(spike_times)):
                if spike_times[i] >= 5000:  # Start analysis at 5 seconds
                    if not in_burst:
                        in_burst = True
                        burst_count += 1
                    duration += 1
                else:
                    if in_burst:
                        in_burst = False

            # Calculate burst frequency
            burst_frequency = burst_count / (simConfig.duration - 5000)  # Duration in ms

           # Save the data for each patient
            output_dir = 'burstdata'
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            with open(os.path.join(output_dir, f'burstdata_patient_{patient}.txt'), 'w') as file:
                file.write(f'Patient {patient}:\n')
                file.write(f'Burst Duration starting at 5 seconds: {duration} ms\n')
                file.write(f'Burst Frequency: {burst_frequency} Hz\n')
            
            # Print the burst duration and frequency
            print(f'Patient {patient}:')
            print(f'Burst Duration starting at 5 seconds: {duration} ms')
            print(f'Burst Frequency: {burst_frequency} Hz')
            
            # Return the burst duration and frequency
            return duration, burst_frequency
        
        simConfig.analysis['analyzeRaster'] = {'func': analyzeRaster, 'kwargs': {'patient': patient}}
        simConfig.filename = 'patient_{0}'.format(patient)
        simConfig.savePickle = True  # Save data in pickle format

        sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)

        
        
        # Create network and run simulation
        sim.createSimulateAnalyze(netParams = netParams, simConfig = simConfig)
        #sim.create(netParams = netParams, simConfig = simConfig)
        
        # Call analyzeRaster function to calculate burst duration and frequency
        analyzeRaster(sim.simData, patient)





        


Start time:  2024-03-07 08:35:32.100964

Creating network of 3 cell populations on 1 hosts...
  Number of cells on node 0: 30 
  Done; cell creation time = 0.00 s.
Making connections...
  Number of connections on node 0: 870 
  Number of synaptic contacts on node 0: 1450 
  Done; cell connection time = 0.09 s.
Adding stims...
  Number of stims on node 0: 100 
  Done; cell stims creation time = 0.01 s.
Recording 60 traces of 2 types on node 0

Running simulation using NEURON for 30000.0 ms...
  Done; run time = 489.71 s; real-time ratio: 0.06.

Gathering data...
  Done; gather time = 0.31 s.

Analyzing...
  Cells: 30
  Connections: 900 (30.00 per cell)
  Synaptic contacts: 1540 (51.33 per cell)
  Spikes: 3636 (4.04 Hz)
  Simulated time: 30.0 s; 1 workers
  Run time: 489.71 s
Saving output as patient_0_data.pkl ... 
Finished saving!
  Done; saving time = 3.72 s.
Plotting recorded cell traces ... cell
Preparing spike data...
Plotting raster...
Unable to run analyzeRaster from sim.plottin

  Number of connections on node 0: 870 
  Number of synaptic contacts on node 0: 1450 
  Done; cell connection time = 0.13 s.
Adding stims...
  Number of stims on node 0: 100 
  Done; cell stims creation time = 0.01 s.
Recording 60 traces of 2 types on node 0

Running simulation using NEURON for 30000.0 ms...
  Done; run time = 552.89 s; real-time ratio: 0.05.

Gathering data...
  Done; gather time = 0.37 s.

Analyzing...
  Cells: 30
  Connections: 900 (30.00 per cell)
  Synaptic contacts: 1540 (51.33 per cell)
  Spikes: 11320 (12.58 Hz)
  Simulated time: 30.0 s; 1 workers
  Run time: 552.89 s
Saving output as patient_3_data.pkl ... 
Finished saving!
  Done; saving time = 4.65 s.
Plotting recorded cell traces ... cell
Preparing spike data...
Plotting raster...
Unable to run analyzeRaster from sim.plotting and sim.analysis. Reason: module 'netpyne.analysis' has no attribute 'analyzeRaster'
  Done; plotting time = 7.72 s

Total time = 565.85 s
Patient 3:
Burst Duration starting at 5 seco